# Software Information Entropy

In [45]:
import datetime
import math
import pathlib
import shutil
import tempfile

import git
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# read example data which includes pubmed github links detected from article abstracts
pd.read_parquet("../../../tests/data/examples/pubmed/pubmed_github_links.parquet")[
    64:84
]

,PMID,article_date,title,authors,github_link
64,34358294,None,Predicting correlated outcomes from molecular ...,"Rauschenberger, Glaab",https://github.com/rauschenberger/joinet
65,36106996,None,Multi-omic integration by machine learning (MI...,"Dickinson, Aufschnaiter, Ott, Meyer",https://github.com/qdickinson/mimal-website
66,36106996,None,Multi-omic integration by machine learning (MI...,"Dickinson, Aufschnaiter, Ott, Meyer",https://github.com/jessegmeyerlab/MIMaL
67,36711640,2023-01-04,IsoAnalytics: A Single-cell Proteomics Web Ser...,"Palmer, Koh, Zhan",https://github.com/zhanxw/Isoplexis_Data_Analysis
68,34755837,None,Deep graph learning of inter-protein contacts.,"Xie, Xu",https://github.com/zw2x/glinter
69,34755837,None,Deep graph learning of inter-protein contacts.,"Xie, Xu",https://github.com/zw2x/glinter/data
70,37015128,2023-08-07,GradMDM: Adversarial Attack on Dynamic Networks.,"Pan, Foo, Zheng, Fan, Rahmani, Ke, Liu",https://github.com/lingengfoo/GradMDM
71,38532317,2024-03-26,COSAP: Comparative Sequencing Analysis Platform.,"Ergun, Cinal, Bakışlı, Emül, Baysan",https://github.com/MBaysanLab/cosap/
72,38532317,2024-03-26,COSAP: Comparative Sequencing Analysis Platform.,"Ergun, Cinal, Bakışlı, Emül, Baysan",https://github.com/MBaysanLab/cosap-webapi/
73,38532317,2024-03-26,COSAP: Comparative Sequencing Analysis Platform.,"Ergun, Cinal, Bakışlı, Emül, Baysan",https://github.com/MBaysanLab/cosap_frontend/


# Lines of Code Changed

In [67]:
def calculate_loc_changes(
    repo_path: pathlib.Path, source: str, target: str, file_names: list[str]
) -> dict[str, int]:
    """
    Finds the total number of code lines changed for each specified file between two commits.

    Args:
        repo_path (pathlib.Path): The path to the git repository.
        source (str): The source commit hash.
        target (str): The target commit hash.
        file_names (list[str]): List of file names to calculate changes for.

    Returns:
        dict[str, int]: A dictionary where the key is the filename, and the value is the lines changed (added and removed).
    """
    repo = git.Repo(repo_path)
    changes = {}

    for file_name in file_names:
        # Get the diff output for the file between the two commits
        diff_output = repo.git.diff(source, target, "--numstat", "--", file_name)
        lines_changed = 0
        for line in diff_output.splitlines():
            diff_line = line.split()
            # Check if the line has exactly three parts: (addded,removed,file_path) and that the first two are numeric
            if (
                len(diff_line) == 3
                and diff_line[0].isdigit()
                and diff_line[1].isdigit()
            ):
                added, removed, _ = diff_line
                lines_changed += int(added) + int(removed)

        changes[file_name] = lines_changed

    return changes

# Normalized Entropy Calculation 

In [85]:
def calculate_normalized_entropy(
    repo_path: pathlib.Path,
    source_commit: str,
    target_commit: str,
    file_names: list[str],
) -> dict[str, float]:
    """
    Calculates the entropy of changes in specified files between two commits,
    inspired by Shannon's entropy formula. Normalized relative to the total lines
    of code changes across specified files.

    Args:
        repo_path (str): The file path to the git repository.
        source_commit (str): The git hash of the source commit.
        target_commit (str): The git hash of the target commit.
        file_names (list[str]): List of file names to calculate entropy for.

    Returns:
        dict[str, float]: A dictionary mapping file names to their calculated entropy.

    Application of Entropy Calculation:
        Entropy measures the uncertainty in a given system. Calculating the entropy
        of lines of code (LoC) changed reveals the variability and complexity of
        modifications in each file. Higher entropy values indicate more unpredictable
        changes, helping identify potentially unstable code areas.

    """
    loc_changes = calculate_loc_changes(
        repo_path, source_commit, target_commit, file_names
    )

    # Calculate total lines of code changes across all specified files
    total_changes = sum(loc_changes.values())

    # Calculate the entropy for each file, relative to total changes
    entropy_calculation = {
        file_name: (
            -(
                (loc_changes[file_name] / total_changes)
                * math.log2(
                    loc_changes[file_name] / total_changes
                )  # Entropy Calculation
            )
            if loc_changes[file_name] != 0
            and total_changes
            != 0  # Avoid division by zero and ensure valid entropy calculation
            else 0.0
        )
        for file_name in loc_changes  # Iterate over each file in loc_changes dictionary
    }
    # Calculate total entropy
    total_entropy = sum(entropy_calculation.values())

    # Normalize total entropy to range [0, 1]
    max_entropy = len(loc_changes)
    normalized_total_entropy = total_entropy / max_entropy

    return normalized_total_entropy

# Proccess Repository

In [88]:
def process_repository(repo_url: str) -> (float, int):
    temp_dir = tempfile.mkdtemp()

    try:
        # Clone the repository
        repo = git.Repo.clone_from(repo_url, temp_dir)
        repo_path = pathlib.Path(temp_dir)

        # Get the default branch
        default_branch = repo.active_branch.name

        # Get the first and most recent commits
        commits = list(repo.iter_commits(default_branch))
        first_commit = commits[-1]
        most_recent_commit = commits[0]

        # Calculate the total existence time (in days)
        time_of_existence = (
            most_recent_commit.committed_datetime - first_commit.committed_datetime
        ).days

        # Find all files that have been edited
        file_names = set()
        for commit in commits:
            for diff in commit.diff(None):
                if diff.a_path:
                    file_names.add(diff.a_path)
                if diff.b_path:
                    file_names.add(diff.b_path)
        file_names = list(file_names)

        # Calculate the total normalized entropy
        normalized_total_entropy = calculate_normalized_entropy(
            repo_path, first_commit.hexsha, most_recent_commit.hexsha, file_names
        )
        return normalized_total_entropy, time_of_existence

    finally:
        # Delete the cloned repository
        shutil.rmtree(temp_dir)

# Main

In [93]:
def main():
    # Read the parquet file
    df = pd.read_parquet(
        "../../../tests/data/examples/pubmed/pubmed_github_links.parquet"
    )

    # Extract the repository URLs from the DataFrame
    repo_urls = df["github_link"].iloc[100:120].tolist()

    entropies = []
    times = []

    for repo_url in repo_urls:
        try:
            normalized_total_entropy, time_of_existence = process_repository(repo_url)
            entropies.append(normalized_total_entropy)
            times.append(time_of_existence)
            print(
                f"Repository: {repo_url}, Normalized Total Entropy: {normalized_total_entropy:.4f}, Time of Existence: {time_of_existence} days"
            )
        except Exception as e:
            print(f"Failed to process repository {repo_url}: {e}")

    # Plotting with Seaborn and Matplotlib
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x=times, y=entropies, s=100, alpha=0.8, color="blue", edgecolor="black"
    )  # Customizing markers
    plt.title("Repository Entropy vs. Time of Existence", fontsize=16)
    plt.xlabel("Time of Existence (days)", fontsize=14)
    plt.ylabel("Normalized Total Entropy", fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    main()

Repository: https://github.com/burtonrj/CytoPyManuscript, Normalized Total Entropy: 0.0000, Time of Existence: 1 days
Repository: https://github.com/PhilippJunk/homelette, Normalized Total Entropy: 0.0919, Time of Existence: 750 days
Repository: https://github.com/Benjamin-Crysup/prosynar, Normalized Total Entropy: 0.0398, Time of Existence: 836 days
Failed to process repository https://github.com/Benjamin-Crysup/prosynar/releases/download/1: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/Benjamin-Crysup/prosynar/releases/download/1 /tmp/tmpb4rlr5vb
  stderr: 'Cloning into '/tmp/tmpb4rlr5vb'...
remote: Not Found
fatal: repository 'https://github.com/Benjamin-Crysup/prosynar/releases/download/1/' not found
'


Pumping 'stdout' of cmd(['git', 'diff', '6998f1576c9e5e421d4b8318fb00e7202f0f456e', '--abbrev=40', '--full-index', '-M', '--raw', '-z', '--no-color']) failed due to: ValueError("Reference at 'refs/heads/main' does not exist")
Exception in thread Thread-64813 (pump_stream):
Traceback (most recent call last):
  File "/home/willdavidson/.pyenv/versions/3.11.0/lib/python3.11/site-packages/git/cmd.py", line 159, in pump_stream
    handler(line)
  File "/home/willdavidson/.pyenv/versions/3.11.0/lib/python3.11/site-packages/git/diff.py", line 769, in <lambda>
    lambda byt: cls._handle_diff_line(byt, repo, index),
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/willdavidson/.pyenv/versions/3.11.0/lib/python3.11/site-packages/git/diff.py", line 730, in _handle_diff_line
    diff = Diff(
           ^^^^^
  File "/home/willdavidson/.pyenv/versions/3.11.0/lib/python3.11/site-packages/git/diff.py", line 441, in __init__
    for submodule in repo.submodules:
                 

KeyboardInterrupt: 